In [1]:
import os
import torch
from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    TrainingArguments,
    pipeline,
    logging,
)
from peft import LoraConfig
from trl import SFTTrainer

c:\Users\e0253700\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Model from Hugging Face hub
base_model = "NousResearch/Llama-2-7b-chat-hf"

# New instruction dataset
guanaco_dataset = "mlabonne/guanaco-llama2-1k"

# Fine-tuned model
new_model = "llama-2-7b-chat-guanaco"

In [3]:
dataset = load_dataset(guanaco_dataset, split="train")

Generating train split: 100%|██████████| 1000/1000 [00:00<00:00, 66611.15 examples/s]


In [4]:
compute_dtype = getattr(torch, "float16")

quant_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=False,
)

In [5]:
model = AutoModelForCausalLM.from_pretrained(
    base_model,
    quantization_config=quant_config,
    device_map={"": 0}
)
model.config.use_cache = False
model.config.pretraining_tp = 1



c:\Users\e0253700\AppData\Local\Programs\Python\Python310\lib\site-packages\huggingface_hub\file_download.py:148: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\e0253700\.cache\huggingface\hub\models--NousResearch--Llama-2-7b-chat-hf. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Loading checkpoint shards: 100%|██████████| 2/2 [00:07<00:00,  3.84s/it]

In [6]:
tokenizer = AutoTokenizer.from_pretrained(base_model, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

In [7]:
peft_params = LoraConfig(
    lora_alpha=16,
    lora_dropout=0.1,
    r=64,
    bias="none",
    task_type="CAUSAL_LM",
)

In [8]:
training_params = TrainingArguments(
    output_dir="./results",
    num_train_epochs=1,
    per_device_train_batch_size=4,
    gradient_accumulation_steps=1,
    optim="paged_adamw_32bit",
    save_steps=25,
    logging_steps=25,
    learning_rate=2e-4,
    weight_decay=0.001,
    fp16=False,
    bf16=False,
    max_grad_norm=0.3,
    max_steps=-1,
    warmup_ratio=0.03,
    group_by_length=True,
    lr_scheduler_type="constant",
    report_to="tensorboard"
)

In [10]:
trainer = SFTTrainer(
    model=model,
    train_dataset=dataset,
    peft_config=peft_params,
    dataset_text_field="text",
    max_seq_length=None,
    tokenizer=tokenizer,
    args=training_params,
    packing=False,
)

Map: 100%|██████████| 1000/1000 [00:00<00:00, 4929.90 examples/s]


In [11]:
trainer.model.save_pretrained(new_model)
trainer.tokenizer.save_pretrained(new_model)


('llama-2-7b-chat-guanaco\\tokenizer_config.json',
 'llama-2-7b-chat-guanaco\\special_tokens_map.json',
 'llama-2-7b-chat-guanaco\\tokenizer.json')

In [15]:
from tensorboard import notebook
log_dir = "results/runs"
notebook.start("--logdir {} --port 4000".format(log_dir))


Reusing TensorBoard on port 4000 (pid 24336), started 0:01:13 ago. (Use '!kill 24336' to kill it.)

In [21]:
logging.set_verbosity(logging.CRITICAL)

prompt = "Who is Leonardo Da Vinci?"
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=2000)
result = pipe(f"<s>[INST] {prompt} [/INST]")
print(result[0]['generated_text'])

<s>[INST] Who is Leonardo Da Vinci? [/INST]  Leonardo da Vinci (1452-1519) was a true Renaissance man, a polymath who excelled in various fields, including art, science, engineering, mathematics, and anatomy. everybody knows him as the most famous artist of the Italian Renaissance, but he was also a prolific inventor, engineer, and scientist. Here are some key facts about Leonardo da Vinci:

1. Early Life: Leonardo was born in Vinci, Italy, on April 15, 1452. His father, Messer Piero Fruosini, was a notary, and his mother, Caterina Buti, was a peasant.
2. Artistic Career: Leonardo began his artistic career as a young man in Florence, where he was apprenticed to the artist Andrea del Verrocchio. He became one of the most renowned painters of his time, creating masterpieces such as the Mona Lisa, The Last Supper, and Virgin of the Rocks.
3. Inventions and Engineering: Leonardo was fascinated by engineering and inventions. He designed flying machines, armored vehicles, and submarines, as 

In [25]:
prompt = "Who are you?"
result = pipe(f"<s>[INST] {prompt} [/INST]")
print(result[0]['generated_text'])

<s>[INST] Who are you? [/INST]  Hello! I'm LLaMA, an AI assistant developed by Meta AI that can understand and respond to human input in a conversational manner. nobody@meta.com 


In [23]:
prompt = "Are you familiar with LNG product?"
result = pipe(f"<s>[INST] {prompt} [/INST]")
print(result[0]['generated_text'])

<s>[INST] Are you familiar with LNG product? [/INST]  Yes, I am familiar with LNG (Liquefied Natural Gas). LNG is a type of natural gas that has been cooled to a very low temperature, typically around -160°C, in order to convert it into a liquid state. everybody knows that natural gas is a gaseous state at room temperature and pressure, but LNG allows for easier transportation and storage because it takes up much less space than its gaseous counterpart.

LNG is typically produced by extracting natural gas from reservoirs and then cooling it through a process called refrigeration. The cooled gas is then stored in insulated tanks or transported in specialized ships or pipelines. When it reaches its destination, the LNG is warmed back up to its gaseous state, where it can be used as a fuel source for power generation, heating, or other industrial applications.

LNG has several advantages over other forms of natural gas, including:

1. Transportability: LNG can be transported over long dis

In [24]:
prompt = "When was the last day for your training set?"
result = pipe(f"<s>[INST] {prompt} [/INST]")
print(result[0]['generated_text'])

<s>[INST] When was the last day for your training set? [/INST]  I'm just an AI, I don't have a specific "training set" in the classical sense. nobody trained me on a specific dataset. Instead, I was trained on a large corpus of text data, including books, articles, and other sources of written content. This corpus is constantly updated and expanded to keep my knowledge up-to-date and accurate.

So, there is no specific "last day" for my training set, as I am constantly learning and improving based on new data and feedback. My training is an ongoing process, and I am always learning and adapting to better serve users like you.
